In [14]:
import pandas as pd
import json
from pandas import json_normalize
from clasificador import *


In [4]:

file_path = 'large_eve.json'
with open(file_path, 'r') as file:
    data = [json.loads(line) for line in file]


total_records = len(data)
total_records


746909

In [5]:
dns_records = [record for record in data if record.get('event_type') == 'dns']

print(f"Cantidad de registros DNS: {len(dns_records)}")

Cantidad de registros DNS: 15749


In [6]:
print("Dos registros DNS aleatorios:")
print(json.dumps(dns_records[:2], indent=4))


Dos registros DNS aleatorios:
[
    {
        "timestamp": "2017-07-22T17:33:16.661646-0500",
        "flow_id": 1327836194150542,
        "pcap_cnt": 22269,
        "event_type": "dns",
        "vlan": 110,
        "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
        "src_port": 59680,
        "dest_ip": "2001:0500:0001:0000:0000:0000:803f:0235",
        "dest_port": 53,
        "proto": "UDP",
        "dns": {
            "type": "query",
            "id": 15529,
            "rrname": "api.wunderground.com",
            "rrtype": "A",
            "tx_id": 0
        }
    },
    {
        "timestamp": "2017-07-22T17:33:24.990320-0500",
        "flow_id": 2022925111925872,
        "pcap_cnt": 54352,
        "event_type": "dns",
        "vlan": 110,
        "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
        "src_port": 38051,
        "dest_ip": "2001:0500:0003:0000:0000:0000:0000:0042",
        "dest_port": 53,
        "proto": "UDP",
        "dns": {
            "type

In [7]:
df_dns = json_normalize(dns_records)
print(f"Forma del DataFrame: {df_dns.shape}")
df_dns.head()

Forma del DataFrame: (15749, 18)


,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN


In [8]:
df_dns_a = df_dns[df_dns['dns.rrtype'] == 'A']
print(f"Registros tipo A: {len(df_dns_a)}") 

Registros tipo A: 2849


In [9]:
unique_domains = df_dns_a['dns.rrname'].unique()
print(f"Dominios únicos tipo A: {len(unique_domains)}")

Dominios únicos tipo A: 177


In [10]:
import tldextract

def obtener_tld(domain):

    resultado = tldextract.extract(domain)
    tld = resultado.domain
    
    if resultado.suffix:
        tld += '.' + resultado.suffix

    return tld

print(obtener_tld("api.wunderground.com"))  
print(obtener_tld("safebrowsing.clients.google.com.home")) 

wunderground.com
home


In [11]:
tlds = [obtener_tld(domain) for domain in unique_domains]

tlds_df = pd.DataFrame(tlds, columns=['domain_tld'])
tlds_df.head()

,domain_tld
0,wunderground.com
1,dropbox.com
2,aoltw.net
3,home
4,mozilla.com


In [16]:
import pickle
import sklearn


print("Current scikit-learn version:", sklearn.__version__)

Pkl_Filename = "Pickle_RL_Model.pkl"

try:
    with open(Pkl_Filename, 'rb') as file:
        Pickled_LR_Model = pickle.load(file)
except ValueError as e:
    print("Error loading the pickle file:", e)

Current scikit-learn version: 1.4.1.post1
Error loading the pickle file: node array from the pickle has an incompatible dtype:
- expected: {'names': ['left_child', 'right_child', 'feature', 'threshold', 'impurity', 'n_node_samples', 'weighted_n_node_samples', 'missing_go_to_left'], 'formats': ['<i8', '<i8', '<i8', '<f8', '<f8', '<i8', '<f8', 'u1'], 'offsets': [0, 8, 16, 24, 32, 40, 48, 56], 'itemsize': 64}
- got     : [('left_child', '<i8'), ('right_child', '<i8'), ('feature', '<i8'), ('threshold', '<f8'), ('impurity', '<f8'), ('n_node_samples', '<i8'), ('weighted_n_node_samples', '<f8')]


In [22]:
df = clasificacion(tlds_df)
df.head()

ValueError: node array from the pickle has an incompatible dtype:
- expected: {'names': ['left_child', 'right_child', 'feature', 'threshold', 'impurity', 'n_node_samples', 'weighted_n_node_samples', 'missing_go_to_left'], 'formats': ['<i8', '<i8', '<i8', '<f8', '<f8', '<i8', '<f8', 'u1'], 'offsets': [0, 8, 16, 24, 32, 40, 48, 56], 'itemsize': 64}
- got     : [('left_child', '<i8'), ('right_child', '<i8'), ('feature', '<i8'), ('threshold', '<f8'), ('impurity', '<f8'), ('n_node_samples', '<i8'), ('weighted_n_node_samples', '<f8')]